# Proyecto 1 Inteligencia de negocios - Clasificación Fake News #

Grupo 17 - Miembros:

*   Juan Esteban Diaz Castro - 202121646
*   Jose Botello - 20
*   Samuel Ramirez - 20


In [17]:
%pip install numpy
%pip install pandas
%pip install joblib
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install scipy
%pip install jupyter

%pip install spacy
!python -m spacy download es_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe: No module named spacy


In [18]:
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

import spacy
import sys

!{sys.executable} -m spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\juane\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [38]:
nlp = spacy.load("es_core_news_sm")

# 1. Carga de Datos

In [19]:
data = "./data/fake_news_spanish.csv"

news = pd.read_csv(data, sep=";", encoding="ISO-8859-1")

data_news = news.copy()

data_news.shape

(57063, 5)

In [20]:
data_news.sample(5)

,ID,Label,Titulo,Descripcion,Fecha
33158,ID,1,La Generalitat Valenciana propone por primera ...,"La Generalitat, a travÃ©s de la Conselleria de...",29/11/2020
36029,ID,0,Cs pasa pÃ¡gina de su crisis interna y afianza...,Fuentes de la confianza de Alfonso Guerra admi...,01/07/2019
17870,ID,1,La PolicÃ­a Nacional balear tapÃ³ las denuncia...,El primero que denunciÃ³ la participaciÃ³n de ...,02/10/2019
54062,ID,0,DÃ­az enfrÃ­a a Alfonso Guerra: 'Anda nervioso...,"La presidenta, en un vÃ­deo subido en su cuent...",08/09/2018
47989,ID,0,Carmen de la PeÃ±a: âEl seÃ±or X del Iniciat...,El director de El EspaÃ±ol ha cantado las cuar...,23/01/2018


In [21]:
data_news.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

# 2. Perfilamiento de Datos

Separación datos en train y test

In [22]:
X_train, X_test, y_train , y_test = train_test_split(data_news, data_news["Label"], test_size=0.3, random_state=1)

X_train.shape, y_train.shape

((39944, 5), (39944,))

In [23]:
X_test.shape, y_test.shape

((17119, 5), (17119,))

In [24]:
X_train

,ID,Label,Titulo,Descripcion,Fecha
23862,ID,1,El Ayuntamiento de Barcelona reclama 'mano ten...,"Las cuentas ascienden a 3.600 millones, una ci...",14/10/2022
44273,ID,0,El EQUO vuelve a estar por encima del 30% en e...,Los de David de la Cruz y los de Pablo MÃ³nica...,30/01/2020
43156,ID,1,"MÃ¡s Madrid, Podemos y el Ayuntamiento de Madr...","JosÃ© Luis MartÃ­nez-Almeida, dice que quiere ...",08/04/2022
13087,ID,0,La presencia de mujeres en las cÃºpulas de los...,"EAJ-PNV y PNC, los dos partidos mÃ¡s crÃ­ticos...",07/03/2023
56295,ID,0,Las defensas intentan frenar que el caso ERE s...,Los abogados de los 22 ex altos cargos del Gob...,10/01/2018
...,...,...,...,...,...
50057,ID,0,Encuesta: Â¿CÃ³mo valora las penas a la cÃºpul...,El Supremo condena por sediciÃ³n a la cÃºpula ...,10/10/2019
32511,ID,0,Cristina Narbona insta a evitar el bloqueo y q...,Cualquier opciÃ³n es bienvenida. La opciÃ³n qu...,05/10/2019
5192,ID,1,La patronal reconoce que hay relaciÃ³n laboral...,El Ministerio de Trabajo y agentes sociales co...,10/02/2021
12172,ID,1,Aguado elige al jefe del Canal de Isabel II co...,"Rafael MartÃ­n Prieto, se sitÃºa como el princ...",30/08/2019


In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39944 entries, 23862 to 33003
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39944 non-null  object
 1   Label        39944 non-null  int64 
 2   Titulo       39930 non-null  object
 3   Descripcion  39944 non-null  object
 4   Fecha        39944 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [26]:
X_train.count()

ID             39944
Label          39944
Titulo         39930
Descripcion    39944
Fecha          39944
dtype: int64

In [27]:
X_train.describe()

,Label
count,39944.000000
mean,0.580988
std,0.493403
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [29]:
X_train["Descripcion"].apply(len).describe()

count    39944.000000
mean       248.101793
std        282.318642
min         37.000000
25%        175.000000
50%        226.000000
75%        272.000000
max      14281.000000
Name: Descripcion, dtype: float64

2.2 Análisis de Calidad

2.2.1 Completitud

In [30]:
df_porcentajes = (100*X_train.isna().sum()/len(X_train)).to_frame()
df_porcentajes.sort_values(0, ascending = False)

,0
Titulo,0.035049
ID,0.000000
Label,0.000000
Descripcion,0.000000
Fecha,0.000000


In [31]:
filas_incompletas = X_train[X_train.isna().any(axis=1)]
filas_incompletas


,ID,Label,Titulo,Descripcion,Fecha
3243,ID,0,NaN,"Hace unos 75 aÃ±os, Hermann GÃ¶ring testificÃ³...",16/06/2019
36724,ID,0,NaN,"Como muchos de ustedes recordarÃ¡n, ayer por l...",04/02/2023
55146,ID,0,NaN,Denuncia PÃºblica\nSe veÃ­a venir... En Gualac...,15/02/2023
47967,ID,0,NaN,Se han dado a conocer los datos electorales pr...,19/04/2019
7345,ID,0,NaN,Defienden dichos con capitulos de Don Gato.\n\...,02/07/2023
7652,ID,0,NaN,Lean (y vean la imagen) con mucha atenciÃ³n:\n...,04/08/2018
4189,ID,0,NaN,"Evidentemente, Barak Obama ha sido arrestado e...",10/03/2022
5041,ID,0,NaN,"""DespuÃ©s de convertirme en presidente, le ped...",16/03/2022
35816,ID,0,NaN,#ÃLTIMA_HORA Le ordenarÃ© a los empresarios d...,07/05/2021
15159,ID,0,NaN,Declarar a Bill Gates enemigo pÃºblico y proce...,02/08/2024


Podemos eliminar los registros que estan vacios

In [32]:
def drop_empty(train):

    train.dropna(inplace=True)

    return

print(drop_empty(train=X_train))

X_train.shape

None


(39930, 5)

In [33]:
filas_incompletas = X_train[X_train.isna().any(axis=1)]
filas_incompletas

,ID,Label,Titulo,Descripcion,Fecha


2.2.2 Duplicidad

In [34]:
duplicated_rows = X_train.loc[X_train.duplicated(subset=["Titulo"]+["Descripcion"], keep=False)].sort_values("Titulo")
duplicated_rows

,ID,Label,Titulo,Descripcion,Fecha
14575,ID,1,"'Hazlo por Madrid', el lema del PSOE se centra...",Gabilondo asegura que si hay movilizaciÃ³n Ayu...,12/04/2021
40673,ID,1,"'Hazlo por Madrid', el lema del PSOE se centra...",Gabilondo asegura que si hay movilizaciÃ³n Ayu...,12/04/2021
31439,ID,1,'SÃ¡lvame Deluxe' entra en polÃ­tica,El presentador del programa de Telecinco acomp...,24/04/2021
13847,ID,1,'SÃ¡lvame Deluxe' entra en polÃ­tica,El presentador del programa de Telecinco acomp...,24/04/2021
50738,ID,1,70.000 millones en tres aÃ±os para modernizar ...,El 39% de los fondos se destinarÃ¡ a la transi...,13/04/2021
...,...,...,...,...,...
28964,ID,1,Â¿A quÃ© hora puedo ir a votar en las eleccion...,La Comunidad de Madrid ha establecido una guÃ­...,30/04/2021
54584,ID,1,Â¿Hartazgo de centralismo en ValÃ¨ncia?,La guerra de declaraciones entre Ayuso y el go...,15/04/2021
54615,ID,1,Â¿Hartazgo de centralismo en ValÃ¨ncia?,La guerra de declaraciones entre Ayuso y el go...,15/04/2021
10974,ID,1,Â¿Ignorarles o rebatirles?: la nueva campaÃ±a ...,La formaciÃ³n de Abascal busca normalizar las ...,21/04/2021


Eliminar noticias duplicadas

In [35]:
def drop_duplicates(train):

    train.drop_duplicates(subset = ['Titulo'] + ['Descripcion'], keep = 'first', inplace = True)

    duplicated_rows = data_news.loc[data_news.duplicated(subset=["Titulo"]+["Descripcion"], keep=False)].sort_values("Titulo")
    print(duplicated_rows)

    return

print(drop_duplicates(train=X_train))

X_train.shape

       ID  Label                                             Titulo  \
40673  ID      1  'Hazlo por Madrid', el lema del PSOE se centra...   
14575  ID      1  'Hazlo por Madrid', el lema del PSOE se centra...   
13847  ID      1               'SÃ¡lvame Deluxe' entra en polÃ­tica   
31439  ID      1               'SÃ¡lvame Deluxe' entra en polÃ­tica   
14981  ID      1  'Vivir a la madrileÃ±a': la realidad no tan id...   
...    ..    ...                                                ...   
8464   ID      1  Â¿Por quÃ© las energÃ­as verdes estÃ¡n causand...   
37769  ID      1  Â¿QuÃ© son las polÃ­ticas activas de empleo y ...   
28272  ID      1  Â¿QuÃ© son las polÃ­ticas activas de empleo y ...   
35301  ID      1  Ãngels BarcelÃ³: 'Los demÃ³cratas somos mucho...   
29781  ID      1  Ãngels BarcelÃ³: 'Los demÃ³cratas somos mucho...   

                                             Descripcion       Fecha  
40673  Gabilondo asegura que si hay movilizaciÃ³n Ayu...  12/04/2021  
14575

(39706, 5)

In [36]:
duplicated_rows = X_train.loc[X_train.duplicated(subset=["Titulo"]+["Descripcion"], keep=False)].sort_values("Titulo")
duplicated_rows

,ID,Label,Titulo,Descripcion,Fecha


2.2.3 Consistencia

In [ ]:
len_titulo = X_train["Titulo"].apply(len)
len_descripcion = X_train["Descripcion"].apply(len)

print("Longitud del Titulo: \n" + str(len_titulo))
print("\nLongitud de la Descripcion: \n" + str(len_descripcion))


Longitud del Titulo: 
23862    109
44273    112
43156    117
13087    130
56295     78
        ... 
50057     62
32511     97
5192     111
12172     85
33003     84
Name: Titulo, Length: 39706, dtype: int64

Longitud de la Descripcion: 
23862    237
44273    273
43156    263
13087    142
56295    330
        ... 
50057    104
32511    160
5192     366
12172    225
33003    191
Name: Descripcion, Length: 39706, dtype: int64
count    39706.000000
mean        96.482068
std         22.079494
min         13.000000
25%         81.000000
50%         96.000000
75%        113.000000
max        258.000000
Name: Titulo, dtype: float64


count    39706.000000
mean       247.504861
std        278.800589
min         37.000000
25%        175.000000
50%        226.000000
75%        272.000000
max      14281.000000
Name: Descripcion, dtype: float64

In [48]:
len_titulo.describe()

count    39706.000000
mean        96.482068
std         22.079494
min         13.000000
25%         81.000000
50%         96.000000
75%        113.000000
max        258.000000
Name: Titulo, dtype: float64

In [49]:
len_descripcion.describe()

count    39706.000000
mean       247.504861
std        278.800589
min         37.000000
25%        175.000000
50%        226.000000
75%        272.000000
max      14281.000000
Name: Descripcion, dtype: float64

2.2.4 Validez

In [101]:
X_train.describe()

,Label
count,39706.000000
mean,0.579333
std,0.493672
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [39]:
X_train["Fecha"] = pd.to_datetime(X_train["Fecha"])

C:\Users\juane\AppData\Local\Temp\ipykernel_32104\1149166573.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  X_train["Fecha"] = pd.to_datetime(X_train["Fecha"])


In [40]:
X_train["Fecha"].value_counts()

Fecha
2023-05-29    67
2019-05-27    58
2019-04-24    54
2019-07-18    52
2019-06-05    50
              ..
2024-09-09     1
2023-08-12     1
2017-02-11     1
2023-11-21     1
2017-08-22     1
Name: count, Length: 2198, dtype: int64